# TP4 - Inter Process Comunication
## Sistemas Operativos

![Logo](static/unpsjb.png)


# Antes de empezar

## Esta guía está versionada con Git

Ésta guía se encuentra en un repositorio Git en [Github](https://github.com/) y se puede descargar con el siguiente comando:

```bash

git clone https://github.com/UNPSJB/so-tp4-2015.git

```

## Versión online
Existe una [versión online](http://nbviewer.ipython.org/format/slides/github/UNPSJB/so-tp4-2015/blob/master/SO_TP4_Guia.ipynb#/), si hubiese cambios, se verían reflejados en ese enlace.


# IPC y visualización con Processing.py

En este práctico, cada grupo resolverá un problema clásico de concurrencia (**Filsofos Comensales**, **Barbero Dormilón**, **La panadería**, **Fumadores**, etc), y generará una interfase visual que represente en pantalla lo que está sucediendo a cada momento.

### Herramientas

 - Procssing.py
     - Visualización
 - netcat
     - Convertir la salida estandar de un proceso en comunicación con otro proceso mediante TCP
 - printf
     - Generar JSON *a mano* con la ayuda de %d (enteros), %f (flotantes), %s (cadenas), etc.


In [132]:
%%file static/esquema.dot
// Esquema graphviz del ejemplo de más abajo
digraph G {
    subgraph cluster_0 {
        // node[shape=square];
        style=filled;
		color=lightgrey;
        prog_a[label="prog_a "];
        prog_b[label="prog_b "];
        prog_c[label="prog_c "];
        prog_d[label="prog_d "];
        label = "procesos de simulacion en C";
     }
    subgraph cluster_1 {
        label="Processing.py"
        th0 [label="Hilo espera clientes/accept()"]
        th1 [label="Hilo 1", color="red"];
        th2 [label="Hilo 2", color="red"];
        th3 [label="Hilo 3", color="red"];
        th4 [label="Hilo 4", color="red"];
        vis [label="Visualización"];
    }
    prog_a -> th1 [label=" netcat"];
    prog_b -> th2 [label=" netcat"];
    prog_c -> th3 [label=" netcat"];
    prog_d -> th4 [label=" netcat"];
    
    th1 -> vis;
    th2 -> vis;
    th3 -> vis;
    th4 -> vis;
    
    th0->th1;
    th0->th2;
    th0->th3;
    th0->th4;    
}

Overwriting static/esquema.dot


In [133]:
from IPython.display import *
!dot -Tpng static/esquema.dot -o static/esquema.png

![esquema](static/esquema.png?asf)
<!-- Image('static/esquema.png') -->

# Processing

Processing es un entorno de desarrollo utilizado para realizar visualización.

Processing tiene una interfase muy similar a la de Arduino y suele ser utilizado como complemento de visualización para sistemas embebidos libres.

![](static/processing_example.png)


## Programación básica en Processing

&nbsp;

```java

void setup() {
    size(500, 500);
}

void loop() {
    rect(10, 10, 20, 20);
}
```


Existen muchas funciones para dibujar en pantalla en la [Referencia](https://processing.org/reference/)

Si bien la librería estandard de Java brinda Sockets con los que nos podemos comunicar con C, no existe una forma fácil de hacerlo con un mecanismo estructurado. El formato JSON que vimos en el útlimo punto del **TP1** es una forma atractiva por ser entendible para humanos y máquinas.

# Processing.py

- Desarollado para facilitar la programación mediante el lenguaje Python
- Python es un lenguaje con baterias incluidas (en este caso vamos a utilizar JSON)
- Todas las funciones de Processing Java están disponibles en Processing Python

![Processing.py](static/processing_py.png)

## Instalación

![Instalación](static/install-processing-py.png)

# Instalación (cont.)
![Instalación seleción modo](static/install-processing-2.png)

#Comunicando C con Processing.py
## JSON

JSON es una forma de pasar datos estructurados mediante cadenas. Generalmente se utiliza en la Web y en bases de datos no relacionales. Vamos a utilizar este formato para que sea sencillod depurar nuestra aplicación, en Processing.py es muy sencillo convertir una cadena JSON en estructura de datos del lenguaje.


## Ejemplo de JSON
```json

{
    "entero": 1,
    "floatante": 2.2,
    "cadena": "soy cadena",
    "objeto": {
        "entero": 2,
        "booleano": true
    }
}
```

### ¿Cómo generamos JSON con C?


```c
#include <stdio.h>
int main(void) {
    /* JSON son cadenas, usemos printf */
    printf("{\"pid\": %d, \"estado\": \"%s\"}", 
        getpid(), 
        "iniciando"
    );
    return 0
}
```

In [8]:
%%file src/imprime_json.c
#include <stdio.h>
#include <unistd.h>

int main(void) {
    /* JSON son cadenas, usemos printf */
    printf("{\"pid\": %d, \"estado\": \"%s\"}", 
        getpid(), 
        "iniciando"
    );
    return 0;
}

Overwriting src/imprime_json.c


In [12]:
!gcc src/imprime_json.c -o imprime_json
!./imprime_json | python -m json.tool

{
    "estado": "iniciando",
    "pid": 82349
}


### El JSON saliendo por stdout

```bash

pepe@localhost $ ./programa 
{pid: 1234, "estado": "iniciando"}
prpr@localhost $
```



### Usando el JSON mediante un pipe a un cat de sockets

```bash
pepe@localhost $ ./programa | nc localhost 4455
```



# Recibiendo los datos con Python


```python
import socket
import json

conexion = socket.socket()
conexion.bind(('localhost', 4455)) # Extremo de escucha para nc
conexion.listen(10)                # Backlog de conexiones 
while True:
    cliente, direccion = conexion.accept()  # Espera una conexión de netcat
    while True:
        cadena = cliente.recv(100)
        if not data:
            break
        datos = json.loads(cadena)
        print datos   # Hacer algo útil

```

# Recibiendo datos con processing.py
```python

import socket
from thread import start_new_thread
# Vairable global de conexion
conexion = None

def setup():
    global conexion
    size(500, 500)
    conexion = socket.socket()
    conexion.bind(('localhost', 4455))
    conexion.listen(10)
    start_new_thread(espera_clientes, (conexion, ))

def espera_clientes(conexion):
    while True:
        cliente, direccion = conexion.accept()
        print "Llegó cliente desde %s %d" % direccion
        while True:
            cadena = cliente.recv(100)
            if not cadena:
                print "Se fue el cliente!"
                break
            print "El cliente dijo: %s" % cadena
            
def stop():
    if conexion:
        conexion.close()
    
def draw():
    fill(255, 0, 0)
    rect(10, 10, 20, 20)
```

# Ejemplo 1: Cambiando un color



In [153]:
%%file static/ejemplo.dot
digraph G {
    programa_c [label="echo '{\"r\": 128, \"g\": 128, \"b\": 128}'"];
    pipe[label="|"];
    nc[label="netcat localhost 4455"];
    programa_c -> pipe -> nc -> socket;
    
    socket[shape=oval];

    draw[label="draw():\n\tfill(r, g, b)\n\t"]
}

Overwriting static/ejemplo.dot


In [154]:
!dot -Tpng static/ejemplo.dot -o static/ejemplo.png

![](static/ejemplo.png?asf)

In [13]:
%%bash
mkdir -p src/examples/json_colores
echo "mode.id=jycessing.mode.PythonMode\nmode=Python\n" > src/examples/json_colores/sketch.properties

```python
import socket
from thread import start_new_thread
import json

# Vairable global de conexion
conexion = None

r, g, b = 255, 255, 255 # Blanco

def setup():
    global conexion
    size(500, 500)
    conexion = socket.socket()
    conexion.bind(('localhost', 4455))
    conexion.listen(10)
    start_new_thread(espera_clientes, (conexion, ))

def espera_clientes(conexion):
    global r, g, b
    while True:
        cliente, direccion = conexion.accept()
        print "Llegó cliente desde %s %d" % direccion
        while True:
            cadena = cliente.recv(100)
            if not cadena:
                print "Se fue el cliente!"
                break
            try:
                data = json.loads(cadena)
                r = data["r"]
                g = data["g"]
                b = data["b"]
            except ValueError:
                print "Porblemas con JSON"
            except KeyError as e:
                print "JSON no tiene algun atributo necesario %s" % e
                
            
def stop():
    if conexion:
        conexion.close()
    
def draw():
    fill(r, g, b)
    rect(10, 10, 20, 20)
```

#### Probando dedse terminal

```bash
pepe@localhost $ echo '{"r": 100, "g": 124, "b": 23}' | nc localhost 4455
```




In [159]:
%save SO_TP4_Guia.ipynb
!make slides
!open ./SO_TP4_Guia.slides.html

'' was not found in history, as a file, url, nor in the user namespace.
ipython nbconvert --to slides SO_TP4_Guia.ipynb
[NbConvertApp] Using existing profile dir: u'/Users/nahuel/.ipython/profile_default'
[NbConvertApp] Converting notebook SO_TP4_Guia.ipynb to slides
[NbConvertApp] Support files will be in SO_TP4_Guia_files/
[NbConvertApp] Loaded template slides_reveal.tpl
[NbConvertApp] Writing 233144 bytes to SO_TP4_Guia.slides.html


In [124]:
ipynb = !ls *.ipynb
assert len(ipynb) == 1  # Solo debería haber un ipynb en este carpeta
ipynb = ipynb[0]

def guardar_celda(path_fragment):
    pass
    
    

In [158]:
%save SO_TP4_Guia.ipynb
!make slides
# Commitear
da
!git add -u
!git commit -m "Update $(date)"
!git push origin master

[master b912dfe] Update Fri Apr 24 12:24:11 ART 2015
 2 files changed, 115 insertions(+), 31 deletions(-)


To git@github.com:UNPSJB/so-tp4-2015.git
   d5f917a..b912dfe  master -> master
